In [ ]:
import pandas as pd
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D,Average, Flatten, Dense, Dropout, Activation , Concatenate, Input , BatchNormalization
from keras.optimizers import SGD
from keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras import Model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from collections import Counter

In [ ]:
# load data
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
df_train.columns

In [ ]:
df_test.columns

In [ ]:
# split our data into features & target
trainX = df_train.drop('label', axis=1).values
trainy = df_train['label'].values.reshape(-1,1)

testX = df_test.values

In [ ]:
trainX[:5]

In [ ]:
trainy[:5]

In [ ]:
testX[:5]

In [ ]:
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape))

In [ ]:
# plot first few images
for i in range(9):
    img = trainX[i].reshape(28,28)
    # define subplot
    plt.subplot(330 + 1 + i)
    # plot raw pixel data
    plt.imshow(img)
    
# show the figure
plt.show()

In [ ]:
# plot first few images
for i in range(9):
    img = trainX[i].reshape(28,28)
    # define subplot
    plt.subplot(330 + 1 + i)
    # plot raw pixel data
    plt.imshow(img,cmap=plt.get_cmap('gray') )
    
# show the figure
plt.show()

In [ ]:
# reshape dataset to have a single channel
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))
# one hot encode target values
trainy = to_categorical(trainy)

In [ ]:
# convert from integers to floats
trainX = trainX.astype('float32')
testX = testX.astype('float32')
# normalize to range 0-1
trainX = trainX / 255.0
testX = testX / 255.0

In [ ]:
print(trainX.shape)

In [ ]:
datagen = ImageDataGenerator(
        validation_split = 0.25,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [ ]:
batch_size = 128
Training_data = datagen.flow(trainX,
                             y=trainy,
                            batch_size = batch_size,
                            subset = 'training')

Validation_data = datagen.flow(trainX,
                             y=trainy,
                            batch_size = batch_size,
                            subset = 'validation')

In [ ]:
# label_list = np.array([])

# for i ,(data,label) in enumerate(Training_data):
#     label_number = np.argmax(label, axis=1)
#     label_list = np.concatenate((label_list, label_number), axis=None)   

In [ ]:
# class_weights = class_weight.compute_class_weight('balanced',np.unique(labels_np),label_list) 
# print(class_weights)

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   mode='auto',
                   verbose=1,
                   patience=20)

lrr= ReduceLROnPlateau(monitor='val_loss',
                       factor=0.1, 
                       patience=5, 
                       min_lr=1e-9)

In [ ]:
input_model = Input((trainX.shape[1],trainX.shape[2],trainX.shape[3]))


model1 = Conv2D(32,(5,5), activation='relu')(input_model)
model1 = BatchNormalization()(model1)
model1 = Conv2D(64,(5,5), activation='relu', padding='same')(model1)
model1 = BatchNormalization()(model1)
model1 = MaxPooling2D((2, 2))(model1)
model1 = Conv2D(64,(3,3), activation='relu' ,padding='same')(model1)
model1 = BatchNormalization()(model1)
model1 = Conv2D(128,(3,3), activation='relu' ,padding='valid')(model1)
model1 = BatchNormalization()(model1)
model1 = AveragePooling2D((2, 2))(model1)
model1 = Flatten()(model1)
model1 = Dense(units = 512, activation = 'relu')(model1)
model1 = Dropout(rate = 0.4)(model1)
model1 = BatchNormalization()(model1)
model1 = Dense(units = 20, activation = 'relu')(model1)
model1 = BatchNormalization()(model1)
model1 = Dense(units = 15, activation = 'relu')(model1)
model1 = BatchNormalization()(model1)
output1 = Dense(units = 10, activation = 'softmax')(model1)
#########################################################                          
model2 = Conv2D(32,(4,4), activation='relu')(input_model)  
model2 = BatchNormalization()(model2)
model2 = Conv2D(32,(4,4), activation='relu', padding='same')(model2)
model2 = BatchNormalization()(model2)
model2 = MaxPooling2D((2, 2))(model2)
model2 = Conv2D(32,(3,3), activation='relu', padding='same')(model2) 
model2 = BatchNormalization()(model2)
model2 = Conv2D(32,(3,3), activation='relu', padding='same')(model2) 
model2 = BatchNormalization()(model2)
model2 = Conv2D(64,(2,2), activation='relu' ,padding='same')(model2)
model2 = BatchNormalization()(model2)
model2 = Conv2D(128,(2,2), activation='relu' ,padding='valid')(model2)
model2 = BatchNormalization()(model2)
model2 = AveragePooling2D((2, 2))(model2)
model2 = Flatten()(model2)
model2 = Dense(units = 1024, activation = 'relu')(model2)
model2 = Dropout(rate = 0.5)(model2)
model2 = BatchNormalization()(model2)
model2 = Dense(units = 64, activation = 'relu')(model2)
model2 = BatchNormalization()(model2)
output2 = Dense(units = 10, activation = 'softmax')(model2)
########################################################
model3 = Conv2D(32,(3,3), activation='relu')(input_model)  
model3 = BatchNormalization()(model3)
model3 = Conv2D(32,(3,3), activation='relu', padding='same')(model3)
model3 = BatchNormalization()(model3)
model3 = MaxPooling2D((2, 2))(model3)
model3 = Conv2D(32,(3,3), activation='relu', padding='same')(model3) 
model3 = BatchNormalization()(model3)
model3 = Conv2D(64,(3,3), activation='relu', padding='same')(model3)
model3 = BatchNormalization()(model3)
model3 = Conv2D(64,(2,2), activation='relu' ,padding='valid')(model3)
model3 = BatchNormalization()(model3)
model3 = Conv2D(128,(2,2), activation='relu' ,padding='valid')(model3)
model3 = BatchNormalization()(model3)
model3 = MaxPooling2D((2, 2))(model3)
model3 = Flatten()(model3)
model3 = Dense(units = 512, activation = 'relu')(model3)
model3 = Dropout(rate = 0.2)(model3)
model3 = BatchNormalization()(model3)
model3 = Dense(units = 64, activation = 'relu')(model3)
model3 = BatchNormalization()(model3)
output3 = Dense(units = 10, activation = 'linear')(model3)
########################################################
output = Average()([output1, output2 , output3])
output = Dense(units = 10, activation = 'softmax')(output)

model = Model(inputs= [input_model], outputs=[output])

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
sgd = SGD(lr=0.1, momentum=0.99)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(Training_data, 
                              epochs= 400,
                              validation_data= Validation_data,
                              # class_weight=class_weights,
                              verbose=1,
                              callbacks=[es , lrr])

In [ ]:
model.save_weights("MNIST_weights.h5")

In [ ]:
val_loss = history.history['val_loss']
loss = history.history['loss']

plt.plot(val_loss)
plt.plot(loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Val error','Train error'], loc='upper right')
plt.savefig('plot_error.png')
plt.show()

In [ ]:
val_accuracy = history.history['val_accuracy']
accuracy = history.history['accuracy']

plt.plot(val_accuracy)
plt.plot(accuracy)
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend(['Val accuracy','Train accuracy'], loc='upper right')
plt.savefig( 'plot_accuracy.png')
plt.show()

In [ ]:
pred = model.predict(testX)
pred = pd.DataFrame(pred)
pred['Label'] = pred.idxmax(axis=1)
pred.head(5)

In [ ]:
pred['index'] = list(range(1,len(pred)+1))
pred.head()

In [ ]:
submission = pred[['index','Label']]
submission.head()

In [ ]:
submission.rename(columns={'index':'ImageId'},inplace = True)
submission.head()

In [ ]:
submission.to_csv('submission.csv',index=False)